# Step 1 Moving to a vertex:

In [5]:
import sympy
def fun(A,t,b,n,z): # if rank of tight rows less than n
    p = float('inf')
    n_inf = float('-inf')
    t_null = sympy.Matrix(t).nullspace() 
    print("null of t : ",t_null)
    lst=np.zeros((len(t_null),n))

    for i in range(len(t_null)):
        lst[i]=np.squeeze(np.asarray(t_null[i]))

    alp={} 

    for i in lst:
        print("trc : ",i)
        A_i = A@i
        t_1 = np.squeeze(np.asarray(A)) 
        t_2 = np.squeeze(np.asarray(b.T-t_1.T))  

        final = np.divide(t_2,A_i)
        max_f=0   
        for j in final:
            if (j)!=p:
                if j > max_f:
                    max_f=j
        min_f = min(final)

        if abs(min_f)>abs(max_f):
            j=10
            print("Optimal solution do not exist")

        print("max : ",max_f)
        alp[max_f]=i

    rank = np.linalg.matrix_rank(t)
    if len(alp)>0:
        alpha=max(alp.keys())
        if alpha ==0:
            print("No Alpha : ",alpha)
            return
        else:
            print("alpha : ",alpha)
            print("vector : ",alp.get(alpha))
            y = alp.get(alpha)
            z_up = np.zeros((n,1))+z

            y = y.reshape((n,1))
        
            z_up = z_up +(y*alpha)
        
            return z_up
    else:
        return
        

In [13]:

import numpy as np
import sys
# from sympy import Matrix 
from scipy.linalg import solve
from scipy.linalg import null_space

# check:  if Ax<=B, A= mat1, B = mat2
def Satisfy (A_dash,B_dash,x):
	delta=pow(10,-10)
	res = np.dot(A_dash,x)
	B_dash=B_dash+delta
	chk=np.less_equal(res, B_dash)
	K=(chk==False)
	if chk[K].size > 0:
		return False
	else:
		return True  

def tgt_constraints(A,B,x,null_s,C,alpha):
	delta=pow(10,-8)
	d=0
	c=0
	while(c<C):
		t=x
		x=x+(alpha*null_s)
		d=d+1
		cons=Satisfy (A,B,x)
		if(d>10000):
			x=np.NaN
			break

		if cons==True:
			u=np.asarray(A @ x)
			sub=abs(np.subtract(B,u))
			chk=sub < delta
			K=(chk==True)    
			c=chk[K].size
		else:
			x=t
			alpha=alpha/10
	return x

def matrix(r,c):
    print("Enter the elements row wise in a single line:") 
    val = list(map(float, input().split())) 
    mat = np.array(val).reshape(r, c)
    return mat

def main():

	# Enter the number of rows and columns and corresponding entries of matrix A, B and C

	m = int(input("Enter the number of rows: "))
	n = int(input("Enter the number of columns: "))

	print("Enter matrix A elements ")
	A=matrix(m,n)
	aa=-1*np.identity(n)
	A=np.concatenate((A,aa))
	print(A)

	print("Enter matrix B elements ")
	B=matrix(m,1)
	col=np.zeros((n,1))
	B=np.vstack([B,col])
	print(B)

	print("Enter matrix C elements ")
	C = matrix(n,1)
	print(C)
    
	print("Enter the feasable point")
	elements = list(map(float, input().split()))
	x = np.array(elements).reshape(n,1)
	print(x)
    

	f_p=0
	m+=n
	f=0
    
	delta=pow(10,-8)
	lc=0
	alpha = 1
	print("x:=",x)
    
	while 1:
		lc=lc+1
		if f==1 and f_p==0:
			x=tgt_constraints(A,B,x,null_s,n,alpha)
		if(f==1 and np.any(np.isnan(x))):
			print("Unbounded soultion")
			break
		f=1
		f_p=0
		print("in main returned x=",x)
		E_dash=np.zeros((1,n))
		B_dash=np.zeros((1,1))
		b_calc=np.asarray(A @ x)
		sub=np.subtract(B,b_calc)
		chk= sub < delta
		b=(chk==True)    
		c=chk[b].size
		count=0
		for i in range(m):
			if(chk[i]==True):
				count=count+1
				row=A[i]
				E_dash= np.vstack([E_dash,row])
				row_n=B[i]
				B_dash=np.vstack([B_dash,row_n])
			if count==n:
				break

		E_dash= np.delete(E_dash,(0),axis=0)
		B_dash= np.delete(B_dash,(0),axis=0)

		print("E_dash:= ",E_dash)
		print("B_dash:= ",B_dash)

		E_dash_inverse = np.linalg.inv(E_dash)			
		print("E_dash_inverse:",E_dash_inverse)
		y = C.transpose()@E_dash_inverse
		print("y=",y)
		minimum=np.min(y)
		if minimum>=0:							# if all values are greater than 0 then it is feasible point
			print("Optimal point := ")
			x=np.linalg.solve(E_dash,B_dash)
			print(x)
			print("Optimal cost :=")
			print(x.transpose()@C)
			break
		else:
			k=0
			idx=0
			for i in range (n):           
				if y[0,i]<0:
					idx=k
					print("idx=",idx)
					break
				k=k+1

			null_s= E_dash_inverse[:,[idx]]
			null_s=-1*null_s
			x= np.linalg.solve(E_dash, B_dash)
			print("updated x:",x)

			print("null_space: ",null_space)

if __name__ == "__main__":
	main()



Enter the number of rows: 2
Enter the number of columns: 2
Enter matrix A elements 
Enter the elements row wise in a single line:
1 1 3 7
[[ 1.  1.]
 [ 3.  7.]
 [-1. -0.]
 [-0. -1.]]
Enter matrix B elements 
Enter the elements row wise in a single line:
10 42
[[10.]
 [42.]
 [ 0.]
 [ 0.]]
Enter matrix C elements 
Enter the elements row wise in a single line:
4 5
[[4.]
 [5.]]
Enter the feasable point
10 0
[[10.]
 [ 0.]]
x:= [[10.]
 [ 0.]]
in main returned x= [[10.]
 [ 0.]]
E_dash:=  [[ 1.  1.]
 [-0. -1.]]
B_dash:=  [[10.]
 [ 0.]]
E_dash_inverse: [[ 1.  1.]
 [-0. -1.]]
y= [[ 4. -1.]]
idx= 1
updated x: [[10.]
 [-0.]]
null_space:  <function null_space at 0x7f44e5e51b00>
in main returned x= [[7.]
 [3.]]
E_dash:=  [[1. 1.]
 [3. 7.]]
B_dash:=  [[10.]
 [42.]]
E_dash_inverse: [[ 1.75 -0.25]
 [-0.75  0.25]]
y= [[3.25 0.25]]
Optimal point := 
[[7.]
 [3.]]
Optimal cost :=
[[43.]]
